# Creating 2 Tables: Sites for Map, Allocations for Table

#### General Approach:
1) Read in csv sheet of data.  Export fields into new database.
2) Remove bad rows and elements.
3) Fix date datatype.
4) Include ID int value for beneficial use.
5) Fix and remove misisng or nan values for allocatoin flow and volume.
6) Remove duplicates based on AllocationNativeID.
7) Create API link for each siteUUID.
8) Export completed df as processed csv.



#### Files Needed
1) dontopen_SiteLJAllow.csv, 
2) dontopen_AlloLFSite.csv

### Sites LF Allow, for creating Site Points, Popups and Legend Colors on Map Only

In [1]:
#Needed Libaraies
import os
import numpy as np
import pandas as pd

#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/Portal Creation Research"
os.chdir(workingDir)

In [2]:
#Sites LeftJoin Allo Input
SitefileInput = "dontopen_SiteLJAllo.csv"
dfsite = pd.read_csv(SitefileInput)
dfsite

C:\Users\rjame\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,SiteUUID,SiteTypeCV,SiteName,Longitude,Latitude,AllocationAmountID,AllocationOwner,Date,WaterSourceID,WaterSourceName,WaterSourceTypeCV,State,WaDENameBenUse,WaDENameWS,WaDENameS
0,AZwr_S1,NON-EXEMPT,Groundwater,-111.814054,32.951858,6377730.0,AR SACATON LLC,NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal
1,AZwr_S10,EXEMPT,Groundwater,-110.231413,31.346347,6448395.0,RICHARD RICHARDS,NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal
2,AZwr_S100,NON-EXEMPT,Groundwater,-112.662912,34.618811,6433985.0,BARBARA E CLARK,NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal
3,AZwr_S1000,EXEMPT,Groundwater,-112.695148,33.946254,6444845.0,"CRAIG, CHARLES ET AL,",NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal
4,AZwr_S10000,MONITOR,Groundwater,-110.917067,32.218850,6192284.0,CITY OF TUCSON - TUCSON WATER,NaN,710000.0,Unspecified,Surface Water,AZ,Other,Surface Water,Monitoring Well
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182659,WYwr_S9995,Spring,BLAIRS NORTH SUPPLY DITCH,-109.019069,41.410619,5581422.0,"BLAIR, ARCHIBALD",1908-01-04,692627.0,Snow Spring,Surface Water,WY,Unknown,Surface Water,Spring
2182660,WYwr_S9996,Spring,FAULKNER DITCH,-109.338400,41.281844,5580546.0,"JOHNSON, W A",1882-12-31,692684.0,Spring Creek,Surface Water,WY,Livestock,Surface Water,Spring
2182661,WYwr_S9997,Spring,EAST AND WEST DITCH,-108.522636,41.387744,5582443.0,"TAYLOR, ROBERT",1905-11-11,693653.0,"Antelope Springs 1, 2, & 3",Surface Water,WY,Livestock,Surface Water,Spring
2182662,WYwr_S9998,Spring,HENRY BROOKS NO. 2 DITCH,-109.094578,41.307778,5584864.0,"BROOKS, JOSEPH",1904-11-09,693236.0,Spring on Head of Pretty Water Creek,Surface Water,WY,Unknown,Surface Water,Spring


In [3]:
#removeing bad rows of df because they are null for some odd reason
dfsite = dfsite[(dfsite.WaterSourceName != 'test')]
dfsite = dfsite[dfsite['SiteUUID'].notna()]
# dfsite = dfsite[dfsite['AllocationPriorityDateID'].notna()]
dfsite = dfsite[dfsite['WaDENameBenUse'].notna()]
dfsite = dfsite[dfsite['Longitude'].notna()]
dfsite = dfsite[dfsite['Latitude'].notna()]
dfsite.reset_index()

,index,SiteUUID,SiteTypeCV,SiteName,Longitude,Latitude,AllocationAmountID,AllocationOwner,Date,WaterSourceID,WaterSourceName,WaterSourceTypeCV,State,WaDENameBenUse,WaDENameWS,WaDENameS
0,0,AZwr_S1,NON-EXEMPT,Groundwater,-111.814054,32.951858,6377730.0,AR SACATON LLC,NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal
1,1,AZwr_S10,EXEMPT,Groundwater,-110.231413,31.346347,6448395.0,RICHARD RICHARDS,NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal
2,2,AZwr_S100,NON-EXEMPT,Groundwater,-112.662912,34.618811,6433985.0,BARBARA E CLARK,NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal
3,3,AZwr_S1000,EXEMPT,Groundwater,-112.695148,33.946254,6444845.0,"CRAIG, CHARLES ET AL,",NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal
4,4,AZwr_S10000,MONITOR,Groundwater,-110.917067,32.218850,6192284.0,CITY OF TUCSON - TUCSON WATER,NaN,710000.0,Unspecified,Surface Water,AZ,Other,Surface Water,Monitoring Well
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699639,2182659,WYwr_S9995,Spring,BLAIRS NORTH SUPPLY DITCH,-109.019069,41.410619,5581422.0,"BLAIR, ARCHIBALD",1908-01-04,692627.0,Snow Spring,Surface Water,WY,Unknown,Surface Water,Spring
1699640,2182660,WYwr_S9996,Spring,FAULKNER DITCH,-109.338400,41.281844,5580546.0,"JOHNSON, W A",1882-12-31,692684.0,Spring Creek,Surface Water,WY,Livestock,Surface Water,Spring
1699641,2182661,WYwr_S9997,Spring,EAST AND WEST DITCH,-108.522636,41.387744,5582443.0,"TAYLOR, ROBERT",1905-11-11,693653.0,"Antelope Springs 1, 2, & 3",Surface Water,WY,Livestock,Surface Water,Spring
1699642,2182662,WYwr_S9998,Spring,HENRY BROOKS NO. 2 DITCH,-109.094578,41.307778,5584864.0,"BROOKS, JOSEPH",1904-11-09,693236.0,Spring on Head of Pretty Water Creek,Surface Water,WY,Unknown,Surface Water,Spring


In [4]:
#AllocationOwner
AllocationOwnerDic = {
"DREAMWORK MANAGEMENT INC; U.S.-FOREST SERVICE; NEL, ALLEN G." : "US Forest Service",
"FOREST SERVICE LINE SPG" : "US Forest Service",
"FOREST SERVICE SPRING PIPELINE" : "US Forest Service",
"FOREST SERVICE,UNITED STATES" : "US Forest Service",
"FOREST SERVICE,UNITED STATES OF AMERICA" : "US Forest Service",
"Forest Service,United States of America -" : "US Forest Service",
"FOREST SERVICE,USDA" : "US Forest Service",
"MAGDALENA RANGER DISTRICT,US Forest Service" : "US Forest Service",
"N.D. Forest Service" : "US Forest Service",
"Rowser and et al.,Lillian C.,USA Forest Service,Rasmussen,Roger J." : "US Forest Service",
"Rowser and et al.,Lillian C.,USA Forest Service,Rasmusson,Roger J.,Hood,E. Ward & Linnea" : "US Forest Service",
"SANTA FE NATIONAL FOREST,USDA FOREST SERVICE" : "US Forest Service",
"State of Utah Division of Wildlife Resources,USA Forest Service" : "US Forest Service",
"Steed,Newel,USA Forest Service" : "US Forest Service",
"TEXAS A&M FOREST SERVICE" : "US Forest Service",
"U S FOREST SERVICE" : "US Forest Service",
"U S Forest Service" : "US Forest Service",
"U. S. FOREST SERVICE" : "US Forest Service",
"U. S. Forest Service, Lassen National Forest, Hat Creek Ranger District" : "US Forest Service",
"U.S. Forest Service" : "US Forest Service",
"U.S. FOREST SERVICE" : "US Forest Service",
"U.S. FOREST SERVICE" : "US Forest Service",
"U.S. FOREST SERVICE; APPLEGATE RANGER DISTRICT" : "US Forest Service",
"U.S. FOREST SERVICE; BEND RANGER DISTRICT" : "US Forest Service",
"U.S. FOREST SERVICE; COLUMBIA RIVER GORGE SCENIC AREA" : "US Forest Service",
"U.S. FOREST SERVICE; MIDDLE FORK RANGER DISTRICT" : "US Forest Service",
"U.S.-FOREST SERVICE" : "US Forest Service",
"U.S.-FOREST SERVICE; U.S.-AGRICULTURE DEPARTMENT" : "US Forest Service",
"UNITED STATES FOREST SERVICE" : "US Forest Service",
"UNITED STATES FOREST SERVICE" : "US Forest Service",
"United States Of America Acting Through Usda Forest Service" : "US Forest Service",
"UNITED STATES OF AMERICA FOREST SERVICE" : "US Forest Service",
"UNITED STATES OF AMERICA, FOREST SERVICE" : "US Forest Service",
"US Forest Service" : "US Forest Service",
"US Forest Service" : "US Forest Service",
"US Forest Service Mount Baker," : "US Forest Service",
"US Forest Service NW International," : "US Forest Service",
"US Forest Service Snoqualmie," : "US Forest Service",
"US Forest Service Wenatchee," : "US Forest Service",
"US Forest Service," : "US Forest Service",
"US Forest Service, LOS PADRES NATIONAL FOREST" : "US Forest Service",
"USA FOREST SERVICE" : "US Forest Service",
"USA Forest Service" : "US Forest Service",
"USA Forest Service - Dixie National Forest" : "US Forest Service",
"USA Forest Service - Dixie National Service" : "US Forest Service",
"USA Forest Service - Fishlake National Forest" : "US Forest Service",
"USA Forest Service / Fishlake National Forest" : "US Forest Service",
"USA Forest Service,Ackerman,Wilford" : "US Forest Service",
"USA Forest Service,Dixie National Forest,USA FOREST SERVICE" : "US Forest Service",
"USA Forest Service,Gardner,Alan Dean,Gardner,Dean C.,Burgess,C. Judd,Gardner,Larry H." : "US Forest Service",
"USA Forest Service,Heirs of E. J. Hancock" : "US Forest Service",
"USA Forest Service,Jones,Richard N." : "US Forest Service",
"USA Forest Service,USA Forest Service 2" : "US Forest Service",
"USA Forest Service,USA Forestt Service" : "US Forest Service",
"USA Forest Service,USA Forsest Service" : "US Forest Service",
"USA Forest Service." : "US Forest Service",
"USA National Forest 2,USA Forest Service" : "US Forest Service",
"USA USDA FOREST SERVICE" : "US Forest Service",
"USDA FOREST SERVICE" : "US Forest Service",
"USDA Forest Service" : "US Forest Service",
"USDA FOREST SERVICE CIBOLA NF" : "US Forest Service",
"USDA FOREST SERVICE LAKE TAHOE BASIN MANAGEMENT UNIT" : "US Forest Service",
"USDA FOREST SERVICE(CARSON NF)" : "US Forest Service",
"USDA Forest Service," : "US Forest Service",
"USDA FOREST SERVICE, SEQUOIA NATIONAL FOREST" : "US Forest Service",
"USDA Forest Service, USA" : "US Forest Service",
"USDA FOREST SERVICE,CARSON NF" : "US Forest Service",
"USDA FOREST SERVICE; DESCHUTES NATIONAL FOREST" : "US Forest Service",
"USDA FOREST SERVICES" : "US Forest Service",
"USDA, FOREST SERVICE,CARSON NF" : "US Forest Service",
"West Panguitch Irrigation and Reservoir Company,USA Forest Service" : "US Forest Service",
"Department of Energy,United States of America" : "US Department of Energy",
"M-K Ferguson Co. for USA Department of Energy" : "US Department of Energy",
"MK-Ferguson Co. for the USA Department of Energy" : "US Department of Energy",
"U S DEPARTMENT OF ENERGY" : "US Department of Energy",
"U.S. Department of Energy" : "US Department of Energy",
"U.S.-DEPARTMENT OF ENERGY,  YUCCA MOUNTAIN SITE CHARACTERIZATION PROJECT" : "US Department of Energy",
"U.S-DEPARTMENT OF ENERGY, YUCCA MOUNTAIN SITE CHARACTERIZATION PROJECT" : "US Department of Energy",
"UNITED STATES DEPARTMENT OF ENERGY" : "US Department of Energy",
"United States of America,U.S. Department of Energy" : "US Department of Energy",
"US Department of Energy," : "US Department of Energy",
"USA Department of Energy Grand Junction Projects" : "US Department of Energy",
"Bankhead,S. Clair,UT Reclamation Mitigation and Conservation Comm,United States of America Bureau of Reclamation,Corbridge,William B." : "US Bureau Reclamation",
"Bryan, Timothy E & Jennifer,Fletcher, Gary & Carla,Teanaway Associates,,Perkins, Charles J & Catherine,US Bureau Reclamation - Yakima," : "US Bureau Reclamation",
"Bureau Of Reclamation" : "US Bureau Reclamation",
"BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"Bureau Of Reclamation," : "US Bureau Reclamation",
"Bureau of Reclamation, Provo Area Office,United States of America,Richardson,Floyd C.,Richardson,W. Earl,Richardson,Jay V.,Sandy City,Richardson,Duane G." : "US Bureau Reclamation",
"Bureau of Reclamation,The Secretary of the Interior" : "US Bureau Reclamation",
"Bureau of Reclamation,United State of America" : "US Bureau Reclamation",
"BUREAU OF RECLAMATION,UNITED STATES D.O.I." : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America" : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America," : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America,Corbridge,William B." : "US Bureau Reclamation",
"BUREAU OF RECLAMATION; UMATILLA FIELD OFFICE" : "US Bureau Reclamation",
"Heart K Ranch,,US Bureau Reclamation - Yakima," : "US Bureau Reclamation",
"Hirst,Paul and Bonnie,Weber River Water Users Association,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Lambert,Paul,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Morgan County,Weber River Water Users` Assocation,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Provo Area Office,U.S.A. Bureau of Reclamation" : "US Bureau Reclamation",
"Provo Area Office,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Remund,Glen & Unice,United States of America Bureau of Reclamation" : "US Bureau Reclamation",
"Smith,Clarence,Siddoway,Benjamin and Beryl,Vernon,James,Vernon,Francis,Gines,Mark E. and Marianne A.,USA Bureau of Reclamation,Smith,Dick,Siddoway,Alan Siddoway and LouDean,Crandall Creek Irrigation Company" : "US Bureau Reclamation",
"U S BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U. S. DEPT. OF THE INTERIOR,BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. Bureau Of Reclamation" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S.A. Bureau of Reclamation" : "US Bureau Reclamation",
"U.S.A. Bureau of Reclamation,Fitzgerald,Jackson R." : "US Bureau Reclamation",
"U.S.-BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"United States Bureau of Reclamation" : "US Bureau Reclamation",
"United States of America Bureau of Reclamation,Spencer Todd and Kelly Colleen Adamson Family Trust" : "US Bureau Reclamation",
"United States of America, Bureau of Reclamation" : "US Bureau Reclamation",
"US BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"US Bureau of Reclamation" : "US Bureau Reclamation",
"US Bureau of Reclamation - Boise," : "US Bureau Reclamation",
"US Bureau Of Reclamation," : "US Bureau Reclamation",
"USA Bureau of Reclamation" : "US Bureau Reclamation",
"USA Bureau of Reclamation -- Provo Area Office" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Boyden,John" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Ivie,James" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Miller,Max" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Weber River Water Users Association,Sanders Properties" : "US Bureau Reclamation",
"UT Reclamation Mitigation and Conservation Comm,United States of America Bureau of Reclamation,Bankhead,S. Clair,Corbridge,William B" : "US Bureau Reclamation",
"Weber River Water Users` Association,Morgan County,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Wilde Revocable Trust,Debra A.,Wilde Land and Livestock Company,Wilde `A` Ditch Company,Faddis,Andrew,Wilde,Henry,Wilson,John,Wilde,Richard & Betty,USA Bureau of Reclamation,Beech,Thomas L.,Wilde Revocable Trust,David Elliot,Wilde,Mary" : "US Bureau Reclamation",
"Wilkinson,Allen,United States of America Bureau of Reclamation" : "US Bureau Reclamation",
"Allen,T. L.,USA Bureau of Reclamation,Eldredge,Edmund,Robinson,Steve B. and Julie Ann,North Chalk Creek Irrigation Company" : "US Bureau Reclamation",
" - Cedar City Field Office,US Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,US Department of Interior, Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,US Department of the Interior, Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,USDI Bureau of Land Management" : "US Bureau of Land Management",
"#NAME?,US Department of Interior, Bureau of Land Management" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"Bureau of Land Management" : "US Bureau of Land Management",
"Bureau of Land Management," : "US Bureau of Land Management",
"Bureau of Land Management, Bakersfield" : "US Bureau of Land Management",
"Bureau of Land Management,Box Elder County" : "US Bureau of Land Management",
"Bureau of Land Management,Davis,Matthew W Davis (II) and Shawnee,Tebbs Living Trust,Richard Ray and Irene H." : "US Bureau of Land Management",
"Bureau of Land Management,Flinspach,Richard and Colleen" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,SOCORRO FIELD OFFICE" : "US Bureau of Land Management",
"Bureau of Land Management,Stuntz Valley Ranch LC" : "US Bureau of Land Management",
"Bureau of Land Management,T Y Cattle Company" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES GOVERNMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES OF AMERICA" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPARTMENT OF INTERIOR" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPT LAND MANAGEMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPT OF INTERIOR" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,USDI" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT; OWYHEE FIELD OFFICE" : "US Bureau of Land Management",
"Carbon County,USA Bureau of Land Management" : "US Bureau of Land Management",
"Cedar City Field Office,USDI Bureau of Land Management;" : "US Bureau of Land Management",
"Chew Livestock Inc,Bureau of Land Management" : "US Bureau of Land Management",
"Department of Interior Bureau of Land Management" : "US Bureau of Land Management",
"Department of Natural Resources,State of Utah,,USA Bureau of Land Management" : "US Bureau of Land Management",
"Division of Wildlife Resources,State of Utah,,USA Bureau of Land Management,Price Field Office" : "US Bureau of Land Management",
"Gemstone Mining Inc.,USA Bureau of Land Management" : "US Bureau of Land Management",
"Grouse Creek Ranch LC,USA Bureau of Land Management" : "US Bureau of Land Management",
"Jordan,W. Clift,USA Bureau of Land Management" : "US Bureau of Land Management",
"Materials Energy Research and Recovery,USA Bureau of Land Management" : "US Bureau of Land Management",
"Moab District USA Bureau of Land Management" : "US Bureau of Land Management",
"Nelson,Rick M.,USA Bureau of Land Management" : "US Bureau of Land Management",
"NORTHERN WASHOE RANCHING MANAGEMENT, LLC; DUCK LAKE RANCH, LLC; U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"NORTHERN WASHOE RANCHING MANAGEMENT, LLC; U.S.-BUREAU OF LAND MANAGEMENT; DUCK LAKE RANCH, LLC" : "US Bureau of Land Management",
"Richfield District USA Bureau of Land Management" : "US Bureau of Land Management",
"Saint George Field Office,USDI Bureau of Land Management" : "US Bureau of Land Management",
"Saint George Field Office,USDI Bureau of Land Management;" : "US Bureau of Land Management",
"Skull Valley Company Ltd.,USA Bureau of Land Management" : "US Bureau of Land Management",
"Staker,James Allen,USA Bureau of Land Management" : "US Bureau of Land Management",
"State of Utah Division of Wildlife Resources,USA Bureau of Land Management" : "US Bureau of Land Management",
"Thousand Peaks Ranches, Inc.,USA Bureau of Land Management,Richfield District" : "US Bureau of Land Management",
"U S BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S. Bureau Of Land Management" : "US Bureau of Land Management",
"U.S. BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S. DEPARTMENT OF THE INTERIOR; BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT; BRIGHT-HOLLAND CORPORATION" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT; WHITE PINE LUMBER CO." : "US Bureau of Land Management",
"U.S.D.I., Bureau of Land Management" : "US Bureau of Land Management",
"United States Of America Acting Through The Department Of Interior Bureau Of Land Management" : "US Bureau of Land Management",
"United States of America Bureau of Land Management,Fillmore District" : "US Bureau of Land Management",
"UNITED STATES OF AMERICA, DEPARTMENT OF THE INTERIOR, BUREAU OF LAND MANAGEMENT; WULFENSTEIN TRUST R-501; SPRING MOUNTAIN MOTORSPORTS RANCH; PAHRUMP GATEWAY UTILITY; SPRING MOUNTAIN RACEWAY LLC" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT / BARSTOW FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT / PALM SPRINGS FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, ALTURAS FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, RIDGECREST FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, UKIAH FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT/ ARCATA FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT/NEEDLES FIELD OFFICE" : "US Bureau of Land Management",
"US DEPT OF THE INTERIOR,BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"USA Bureau of Land Management" : "US Bureau of Land Management",
"USA Bureau of Land Management - Filmore Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management (Richfield District)" : "US Bureau of Land Management",
"USA Bureau of Land Management Vernal District" : "US Bureau of Land Management",
"USA Bureau of Land Management, Richfield District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Cedar City District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Fillmore District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Fillmore Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Kanab District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Kanab Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Moab District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Monticello  District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Monticello District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Ogden District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Price District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Price Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Richfield District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Vernal District" : "US Bureau of Land Management",
"USDI BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"USDI BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"Utah School and Institutional Trust Lands Admin.,USA Bureau of Land Management" : "US Bureau of Land Management",
"West Panguitch Irrigation and Reservoir Company,State of Utah Board of Water Resources,USA Bureau of Land Management" : "US Bureau of Land Management",
"WHITE PINE LUMBER CO; U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"ARMY CAMP SPRING" : "US Army",
"CORPS OF ENGRS. ABIQUIU DAM,US ARMY ENGR DIST ALBQ." : "US Army",
"Cummins, Dale F,US Army Corps Of Engineers," : "US Army",
"Dept Of The Army Corps Of Engineers," : "US Army",
"LONGHORN ARMY AMMUNITION PLANT,U.S. DEPARTMENT OF THE INTERIOR FISH AND WILDLIFE SERVICE" : "US Army",
"U S ARMY CORPS OF ENGINEERS" : "US Army",
"U S Army Corps of Engineers" : "US Army",
"U S Army, Department of" : "US Army",
"U.S Army Corps Of Engineers" : "US Army",
"U.S. ARMY CORP OF ENGINEERS" : "US Army",
"U.S. Army Corps Of Engineers" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS SEATTLE" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS WALLA WALLA" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS, WALLA WALLA DISTRICT" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS; PORTLAND DISTRICT" : "US Army",
"U.S. ARMY CORPS OF ENGINEERSSEATTLE" : "US Army",
"U.S. ARMY CORPS OF ENGINEERSWALLA WALLA" : "US Army",
"U.S. Army Dugway Proving Ground" : "US Army",
"U.S. ARMY WHITE SANDS MISSILE" : "US Army",
"U.S. ARMY-WHITE SANDS MISSILE" : "US Army",
"UNITED STATES ARMY CORPS OF ENGINEERS" : "US Army",
"US ARMY" : "US Army",
"Us Army Corps Of Engineers" : "US Army",
"US ARMY CORPS OF ENGINEERS" : "US Army",
"US Army Corps Of Engineers," : "US Army",
"US ARMY CORPS OF ENGINGEERS" : "US Army",
"US DEPARTMENT OF THE ARMY" : "US Army",
"USA Army Dugway Proving Ground" : "US Army",
"USA Army Dugway Proving Grounds" : "US Army",
"USA Army Engineer District Corps of Engineers" : "US Army",
"USA Army for Dugway Proving Ground" : "US Army",
"USA Army Ogden Sub-Depot of Tooele Ordinance Depot" : "US Army",
"USA Army Tooele Army Depot South" : "US Army",
"USA Department of the Army" : "US Army",
"USA Department of the Army - Dugway Proving Ground" : "US Army",
"USA Department of the Army - Tooele Army Depot" : "US Army",
"USA Department of the Army Arsenal" : "US Army",
"USARMY Corp Of Engineers," : "US Army",
"USARMY Corps Engineers Columbia Cnty," : "US Army",
"USARMY Corps Engineers Seattle," : "US Army",
"USARMY Corps Engineers Walla Walla," : "US Army",
"USARMY Corps Engineers," : "US Army",
"USARMY Corps Engineers,,USDA Agricultural Research Service WRPIS," : "US Army",
"WA Fish & Wildlife Dept - ERO,,USARMY Corps Engineers," : "US Army",
"Camelot International Resort Corporation,United States of America" : "United States of America",
"Christensen,Mark B.,Fish and Wildlife Service,United States of America" : "United States of America",
"CIBOLA NATIONAL FOREST,UNITED STATES OF AMERICA," : "United States of America",
"Ekker,Sheena,Utah School and Institutional Trust Lands Admin.,Hodges,Jimmy,McArthur,Ty,United States of America" : "United States of America",
"Fish and Wildlife Service,United States of America" : "United States of America",
"Fish and Wildlife Service,United States of America," : "United States of America",
"INTERMOUNTAIN LAND CO.; MOONROCKS GATEWAY LLC; UNITED STATES OF AMERICA" : "United States of America",
"LAMB, SHELDON W. & ISABEL H.; UNITED STATES OF AMERICA" : "United States of America",
"Llewellyn,Robert W.,United States of America" : "United States of America",
"R. CLYDE HIIBEL FAMILY TRUST; UNITED STATES OF AMERICA" : "United States of America",
"The United States of America" : "United States of America",
"THE UNITED STATES OF AMERICA IN TRUST FOR THE PYRAMID LAKE PAIUTE TRIBE; GARAVANTA LAND AND LIVESTOCK COMPANY SUCCESSOR TO J. GARAVANTA" : "United States of America",
"THOMAS, DARRELL E.; THOMAS, BEVERLY J.; UNITED STATES OF AMERICA" : "United States of America",
"United States Of America" : "United States of America",
"UNITED STATES OF AMERICA" : "United States of America",
"UNITED STATES OF AMERICA" : "United States of America",
"United States of America" : "United States of America",
"UNITED STATES OF AMERICA (BLM)" : "United States of America",
"United States Of America Acting Through" : "United States of America",
"United States of America in trust for Raymond Murray,Burton,John Ira" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR SUCH SHOSHONE INDIANS SOUTHERN NEVADA" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR THE WASHOE TRIBE OF INDIANS" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR YOMBA TRIBE OF SHOSHONE INDIANS" : "United States of America",
"UNITED STATES OF AMERICA NAS" : "United States of America",
"UNITED STATES OF AMERICA NAVAL AIR STATION FALLON" : "United States of America",
"United States Of America Trustee" : "United States of America",
"United States Of America," : "United States of America",
"UNITED STATES OF AMERICA,  C/O COMMANDER AND DISTRICT ENGINEER,   UNITED STATES ARMY ENGINEER  DISTRICT, LOUISVILLE" : "United States of America",
"United States of America, Utah" : "United States of America",
"United States of America,BBRD, LC,Ottesen,Nellie" : "United States of America",
"United States of America,Owen Probst and Rosetta B. Probst Trust" : "United States of America",
"United States of America,The,Conger,Wm. B. and Florance H" : "United States of America",
"United States of America,Virgin River Land Preservation Inc." : "United States of America",
"UNITED STATES OF AMERICA; STREETS; UNITED STATES OF AMERICA IN TRUST FOR THE RENO SPARKS INDIAN COLONY; NEVADA-DEPARTMENT OF TRANSPORTATION; SIERRA PACIFIC POWER COMPANY; RENO SPARKS INDIAN COLONY; RENO LODGE #13 F AND A M" : "United States of America",
"United States of American" : "United States of America",
"USA-United States Of America" : "United States of America",
"Utah Reclamation Mitigation and Conservation Commission,United States Of America" : "United States of America",
"WASHOE COUNTY; PRATT, TOM & LANNETTE; UNITED STATES OF AMERICA" : "United States of America",
"WS-BUCKLAND, LLC; UNITED STATES OF AMERICA" : "United States of America"}
 
def assignAllocationOwner(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "Other"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = AllocationOwnerDic[String1]
        except:
            outList = "Other"
    return outList

dfsite['AllocationOwner'] = dfsite.apply(lambda row: assignAllocationOwner(row['AllocationOwner']), axis=1)

In [5]:
#Color for WBenUse for the Site Map per point
#Mapdeck uses a build color pacakge called 'colourvalues library', which does use HEX color code, but has to end in an "FF".
WSWCBenUseColorDict = { 
"Agricultural" : "#006400FF",
"Aquaculture" : "#9ACD32FF",
"Commercial" : "#FFFF00FF",
"Domestic" : "#0000FFFF",
"Environmental" : "#32CD32FF",
"Fire" : "#FF4500FF",
"Fish" : "#9370DBFF",
"Flood Control" : "#00FFFFFF",
"Heating and Cooling" : "#FF69B4FF",
"Industrial" : "#800080FF",
"Instream Flow" : "#00BFFFFF",
"Livestock" : "#FFD700FF",
"Mining" : "#A52A2AFF",
"Municipal" : "#4B0082FF",
"Power" : "#FFA500FF",
"Recharge" : "#D2691EFF",
"Recreation" : "#FFC0CBFF",
"Snow Making" : "#F0FFF0FF",
"Storage" : "#F5DEB3FF",
"Wildlife" : "#FF0000FF",
"Other" : "#808080FF",
"Unknown" : "#D3D3D3FF"}
 
def assignWSWCBUColor(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "#D3D3D3FF"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WSWCBenUseColorDict[String1]
        except:
            outList = "#D3D3D3FF"
    return outList

dfsite['WBenUseColor'] = dfsite.apply(lambda row: assignWSWCBUColor(row['WaDENameBenUse']), axis=1)

In [6]:
#Color for SiteType for the Site Map per point
#Mapdeck uses a build color pacakge called 'colourvalues library', which does use HEX color code, but has to end in an "FF".
WSWCSiteTypeColorDict = {
"Abandoned" : "#FFC0CBFF",
"Aquifer" : "#FF1493FF",
"Automated" : "#FFA07AFF",
"Bucket" : "#BDB76BFF",
"Canal / Ditch / Stream" : "#191970FF",
"Creek / Spring" : "#7FFF00FF",
"Dam" : "#008B8BFF",
"Discharge Point" : "#EE82EEFF",
"Diversion Point" : "#BA55D3FF",
"Drain" : "#8A2BE2FF",
"Effluent" : "#800080FF",
"Exchange Plan" : "#483D8BFF",
"Geothermal" : "#AFEEEEFF",
"Gravity" : "#4B0082FF",
"Groundwater" : "#800000FF",
"Lake" : "#F08080FF",
"Legal" : "#4682B4FF",
"Mine" : "#FF0000FF",
"Minimum Flow" : "#FFA500FF",
"Monitoring Well" : "#FF6347FF",
"Other" : "#FFFF00FF",
"Pipeline" : "#FAFAD2FF",
"Point of Rediversion" : "#EEE8AAFF",
"Point of Return" : "#ADFF2FFF",
"Power Plant" : "#32CD32FF",
"Pump" : "#00FF7FFF",
"Reach" : "#228B22FF",
"Recharge Area" : "#9ACD32FF",
"Reservoir" : "#FFE4E1FF",
"Seep" : "#20B2AAFF",
"Sewage" : "#FFEBCDFF",
"Spring" : "#00FFFFFF",
"Storage" : "#B0C4DEFF",
"Sump" : "#48D1CCFF",
"Surface & Groundwater" : "#808000FF",
"Surface Water" : "#00FA9AFF",
"Unknown" : "#D3D3D3FF",
"Well" : "#6495EDFF",
"winter runoff" : "#1E90FFFF"}
 
def assignWSWCSiteTypeColor(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "#D3D3D3FF"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WSWCSiteTypeColorDict[String1]
        except:
            outList = "#D3D3D3FF"
    return outList

dfsite['SiteTypeColor'] = dfsite.apply(lambda row: assignWSWCSiteTypeColor(row['WaDENameS']), axis=1)

In [7]:
#Color for WaterSourceType for the Site Map per point
#Mapdeck uses a build color pacakge called 'colourvalues library', which does use HEX color code, but has to end in an "FF".
WSWCWaterSourceTypeColorDict = {
"Groundwater" : "#008000FF",
"Other" : "#FFA500FF",
"Reservoir / Storage" : "#FFFF00FF",
"Surface & Groundwater" : "#800080FF",
"Surface Water" : "#FF4500FF",
"Unknown" : "#0000FFFF",
"Effluent" : "#FFC0CBFF",
"Lost" : "#8B0000FF",
"Recharge" : "#9370DBFF"}
 
def assignWSWCWaterSourceTypeColor(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "#D3D3D3FF"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WSWCWaterSourceTypeColorDict[String1]
        except:
            outList = "#D3D3D3FF"
    return outList

dfsite['WaterSourceTypeColor'] = dfsite.apply(lambda row: assignWSWCWaterSourceTypeColor(row['WaDENameWS']), axis=1)

In [8]:
#Color for Owner for the Site Map per point
#Mapdeck uses a build color pacakge called 'colourvalues library', which does use HEX color code, but has to end in an "FF".
WSWCOwnerColorDict = { 
"US Forest Service" : "#006400FF",
"US Department of Energy" : "#FFA500FF",
"US Bureau Reclamation" : "#FFFF00FF",
"US Bureau of Land Management" : "#800080FF",
"US Army" : "#FF4500FF",
"United States of America" : "#0000FFFF",
"Other" : "#D3D3D3FF"}
 
def assignWSWCOwnerColor(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "#D3D3D3FF"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WSWCOwnerColorDict[String1]
        except:
            outList = "#D3D3D3FF"
    return outList

dfsite['OwnerColor'] = dfsite.apply(lambda row: assignWSWCOwnerColor(row['AllocationOwner']), axis=1)

In [9]:
#Sort by SiteUUID and BenUse (this way Agriculture should show up first in the below list)
dfsite = dfsite.sort_values(by=['SiteUUID', 'WaDENameBenUse'], ascending=True).reset_index()
dfsite

,index,SiteUUID,SiteTypeCV,SiteName,Longitude,Latitude,AllocationAmountID,AllocationOwner,Date,WaterSourceID,WaterSourceName,WaterSourceTypeCV,State,WaDENameBenUse,WaDENameWS,WaDENameS,WBenUseColor,SiteTypeColor,WaterSourceTypeColor,OwnerColor
0,0,AZwr_S1,NON-EXEMPT,Groundwater,-111.814054,32.951858,6377730.0,Other,NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal,#D3D3D3FF,#4682B4FF,#FF4500FF,#D3D3D3FF
1,1,AZwr_S10,EXEMPT,Groundwater,-110.231413,31.346347,6448395.0,Other,NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal,#D3D3D3FF,#4682B4FF,#FF4500FF,#D3D3D3FF
2,2,AZwr_S100,NON-EXEMPT,Groundwater,-112.662912,34.618811,6433985.0,Other,NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal,#D3D3D3FF,#4682B4FF,#FF4500FF,#D3D3D3FF
3,3,AZwr_S1000,EXEMPT,Groundwater,-112.695148,33.946254,6444845.0,Other,NaN,710000.0,Unspecified,Surface Water,AZ,Unknown,Surface Water,Legal,#D3D3D3FF,#4682B4FF,#FF4500FF,#D3D3D3FF
4,4,AZwr_S10000,MONITOR,Groundwater,-110.917067,32.218850,6192284.0,Other,NaN,710000.0,Unspecified,Surface Water,AZ,Other,Surface Water,Monitoring Well,#808080FF,#FF6347FF,#FF4500FF,#D3D3D3FF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699639,2182659,WYwr_S9995,Spring,BLAIRS NORTH SUPPLY DITCH,-109.019069,41.410619,5581422.0,Other,1908-01-04,692627.0,Snow Spring,Surface Water,WY,Unknown,Surface Water,Spring,#D3D3D3FF,#00FFFFFF,#FF4500FF,#D3D3D3FF
1699640,2182660,WYwr_S9996,Spring,FAULKNER DITCH,-109.338400,41.281844,5580546.0,Other,1882-12-31,692684.0,Spring Creek,Surface Water,WY,Livestock,Surface Water,Spring,#FFD700FF,#00FFFFFF,#FF4500FF,#D3D3D3FF
1699641,2182661,WYwr_S9997,Spring,EAST AND WEST DITCH,-108.522636,41.387744,5582443.0,Other,1905-11-11,693653.0,"Antelope Springs 1, 2, & 3",Surface Water,WY,Livestock,Surface Water,Spring,#FFD700FF,#00FFFFFF,#FF4500FF,#D3D3D3FF
1699642,2182662,WYwr_S9998,Spring,HENRY BROOKS NO. 2 DITCH,-109.094578,41.307778,5584864.0,Other,1904-11-09,693236.0,Spring on Head of Pretty Water Creek,Surface Water,WY,Unknown,Surface Water,Spring,#D3D3D3FF,#00FFFFFF,#FF4500FF,#D3D3D3FF


In [10]:
## Create Sites Dataframe - with attached Allo and Benuse info ##
########################################
outdfSite = pd.DataFrame()

#The Columns
outdfSite['SiteUUID'] = dfsite['SiteUUID'].astype(str) 
outdfSite['Lat'] = dfsite['Latitude'].astype(float) 
outdfSite['Long'] = dfsite['Longitude'].astype(float)
outdfSite['WBenUse'] = dfsite['WaDENameBenUse'].astype(str)
outdfSite['WBenUseColor'] = dfsite['WBenUseColor'].astype(str)
outdfSite['SiteTypeColor'] = dfsite['SiteTypeColor'].astype(str)

outdfSite['State'] = dfsite['State'].astype(str)

outdfSite['WaterSourceTypeColor'] = dfsite['WaterSourceTypeColor'].astype(str)
outdfSite['OwnerColor'] = dfsite['OwnerColor'].astype(str)


#Things we don't actually need for the site csv, but will need for the allowcation.
# outdfSite['SiteTypeCV'] = dfsite['SiteTypeCV'].astype(str)
# outdfSite['WaterSourceTypeCV'] = dfsite['WaterSourceTypeCV'].astype(str)
# outdfSite['AllocationOwner'] = dfsite['AllocationOwner'].astype(str)

#Drop Duplicates - SiteUUID
outdfSite = outdfSite.drop_duplicates(['SiteUUID'], keep="first").reset_index(drop=True)

outdfSite

,SiteUUID,Lat,Long,WBenUse,WBenUseColor,SiteTypeColor,State,WaterSourceTypeColor,OwnerColor
0,AZwr_S1,32.951858,-111.814054,Unknown,#D3D3D3FF,#4682B4FF,AZ,#FF4500FF,#D3D3D3FF
1,AZwr_S10,31.346347,-110.231413,Unknown,#D3D3D3FF,#4682B4FF,AZ,#FF4500FF,#D3D3D3FF
2,AZwr_S100,34.618811,-112.662912,Unknown,#D3D3D3FF,#4682B4FF,AZ,#FF4500FF,#D3D3D3FF
3,AZwr_S1000,33.946254,-112.695148,Unknown,#D3D3D3FF,#4682B4FF,AZ,#FF4500FF,#D3D3D3FF
4,AZwr_S10000,32.218850,-110.917067,Other,#808080FF,#FF6347FF,AZ,#FF4500FF,#D3D3D3FF
...,...,...,...,...,...,...,...,...,...
1277107,WYwr_S9995,41.410619,-109.019069,Unknown,#D3D3D3FF,#00FFFFFF,WY,#FF4500FF,#D3D3D3FF
1277108,WYwr_S9996,41.281844,-109.338400,Livestock,#FFD700FF,#00FFFFFF,WY,#FF4500FF,#D3D3D3FF
1277109,WYwr_S9997,41.387744,-108.522636,Livestock,#FFD700FF,#00FFFFFF,WY,#FF4500FF,#D3D3D3FF
1277110,WYwr_S9998,41.307778,-109.094578,Unknown,#D3D3D3FF,#00FFFFFF,WY,#FF4500FF,#D3D3D3FF


In [11]:
#Exporting Outbound DataFrame to working csv file.
outdfSite.to_csv('P_SiteLJAllo.csv', index=False)  # The output

# Allo LF Site, for filtering and Table Return Only

In [12]:
#Needed Libaraies
import os
import numpy as np
import pandas as pd

#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/Portal Creation Research"
os.chdir(workingDir)

In [13]:
#Allo LeftJoin Sites Input
AllofileInput = "dontopen_AlloLJSite.csv"
dfallo = pd.read_csv(AllofileInput)
dfallo

C:\Users\rjame\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,AllocationAmountID,AllocationNativeID,AllocationFlow_CFS,AllocationVolume_AF,AllocationOwner,Date,SiteUUID,SiteTypeCV,Longitude,Latitude,WaterSourceID,WaterSourceUUID,WaterSourceName,WaterSourceTypeCV,OrganizationID,OrganizationUUID,State,WaDENameBenUse,WaDENameWS,WaDENameS
0,6227330,38-13334.0,0.11,NaN,"HUFF, LARRY R",1971-10-19,AZwr_S217877,Unspecified,-110.772785,33.340579,709978,AZwr_WS5,7,Surface Water,56,ADWR,AZ,Wildlife,Surface Water,Other
1,6227330,38-13334.0,0.11,NaN,"HUFF, LARRY R",1971-10-19,AZwr_S217877,Unspecified,-110.772785,33.340579,709978,AZwr_WS5,7,Surface Water,56,ADWR,AZ,Livestock,Surface Water,Other
2,6273570,38-90944.0,4.80,NaN,AZ STATE LAND DEPT,1950-12-31,AZwr_S217882,Unspecified,-112.076120,33.996216,709983,AZwr_WS10,32,Surface Water,56,ADWR,AZ,Wildlife,Surface Water,Other
3,6273570,38-90944.0,4.80,NaN,AZ STATE LAND DEPT,1950-12-31,AZwr_S217882,Unspecified,-112.076120,33.996216,709983,AZwr_WS10,32,Surface Water,56,ADWR,AZ,Livestock,Surface Water,Other
4,6239656,38-15391.1,2.28,NaN,JOHNSTON'S FUEL LINERS,1950-12-31,AZwr_S217885,Unspecified,-109.128459,31.732190,709986,AZwr_WS13,36,Surface Water,56,ADWR,AZ,Wildlife,Surface Water,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704781,5588956,CR CB03/430,12.44,NaN,NaN,1888-11-08,WYwr_S5751,Stream,-106.748386,43.792047,692704,WYwr_WS154,Willow Creek,Surface Water,10,WWDO,WY,Livestock,Surface Water,Canal / Ditch / Stream
1704782,5587104,CR CC31/096,0.14,NaN,"VOGEL, JAMES",1902-02-24,WYwr_S8536,Spring,-110.799914,43.489072,692713,WYwr_WS1537,Beaver Dam Spring,Surface Water,10,WWDO,WY,Unknown,Surface Water,Spring
1704783,5587124,P27115.0D,2.23,NaN,"GENESIS ALKALI WYOMING, LP",1980-10-02,WYwr_S6868,Stream,-109.801130,41.610980,692722,WYwr_WS806,Snake Draw,Surface Water,10,WWDO,WY,Snow Making,Surface Water,Canal / Ditch / Stream
1704784,5587542,CR CC96/102,4.80,NaN,LITTLE WOLF RANCH INC,1884-12-10,WYwr_S5371,Stream,-107.226603,44.769349,692756,WYwr_WS135,Wolf Creek,Surface Water,10,WWDO,WY,Livestock,Surface Water,Canal / Ditch / Stream


In [14]:
#removeing bad rows of df because they are null for some odd reason
dfallo = dfallo[(dfallo.WaterSourceName != 'test')]
dfallo = dfallo[dfallo['SiteUUID'].notna()]
# dfallo = dfallo[dfallo['AllocationPriorityDateID'].notna()]
dfallo = dfallo[dfallo['WaDENameBenUse'].notna()]
dfallo = dfallo[dfallo['Longitude'].notna()]
dfallo = dfallo[dfallo['Latitude'].notna()]
dfallo.reset_index()

,index,AllocationAmountID,AllocationNativeID,AllocationFlow_CFS,AllocationVolume_AF,AllocationOwner,Date,SiteUUID,SiteTypeCV,Longitude,...,WaterSourceID,WaterSourceUUID,WaterSourceName,WaterSourceTypeCV,OrganizationID,OrganizationUUID,State,WaDENameBenUse,WaDENameWS,WaDENameS
0,0,6227330,38-13334.0,0.11,NaN,"HUFF, LARRY R",1971-10-19,AZwr_S217877,Unspecified,-110.772785,...,709978,AZwr_WS5,7,Surface Water,56,ADWR,AZ,Wildlife,Surface Water,Other
1,1,6227330,38-13334.0,0.11,NaN,"HUFF, LARRY R",1971-10-19,AZwr_S217877,Unspecified,-110.772785,...,709978,AZwr_WS5,7,Surface Water,56,ADWR,AZ,Livestock,Surface Water,Other
2,2,6273570,38-90944.0,4.80,NaN,AZ STATE LAND DEPT,1950-12-31,AZwr_S217882,Unspecified,-112.076120,...,709983,AZwr_WS10,32,Surface Water,56,ADWR,AZ,Wildlife,Surface Water,Other
3,3,6273570,38-90944.0,4.80,NaN,AZ STATE LAND DEPT,1950-12-31,AZwr_S217882,Unspecified,-112.076120,...,709983,AZwr_WS10,32,Surface Water,56,ADWR,AZ,Livestock,Surface Water,Other
4,4,6239656,38-15391.1,2.28,NaN,JOHNSTON'S FUEL LINERS,1950-12-31,AZwr_S217885,Unspecified,-109.128459,...,709986,AZwr_WS13,36,Surface Water,56,ADWR,AZ,Wildlife,Surface Water,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699639,1704781,5588956,CR CB03/430,12.44,NaN,NaN,1888-11-08,WYwr_S5751,Stream,-106.748386,...,692704,WYwr_WS154,Willow Creek,Surface Water,10,WWDO,WY,Livestock,Surface Water,Canal / Ditch / Stream
1699640,1704782,5587104,CR CC31/096,0.14,NaN,"VOGEL, JAMES",1902-02-24,WYwr_S8536,Spring,-110.799914,...,692713,WYwr_WS1537,Beaver Dam Spring,Surface Water,10,WWDO,WY,Unknown,Surface Water,Spring
1699641,1704783,5587124,P27115.0D,2.23,NaN,"GENESIS ALKALI WYOMING, LP",1980-10-02,WYwr_S6868,Stream,-109.801130,...,692722,WYwr_WS806,Snake Draw,Surface Water,10,WWDO,WY,Snow Making,Surface Water,Canal / Ditch / Stream
1699642,1704784,5587542,CR CC96/102,4.80,NaN,LITTLE WOLF RANCH INC,1884-12-10,WYwr_S5371,Stream,-107.226603,...,692756,WYwr_WS135,Wolf Creek,Surface Water,10,WWDO,WY,Livestock,Surface Water,Canal / Ditch / Stream


In [15]:
#changing data type of 'Date' to datatype, then changing format of 'Date' to yyyy-mm-dd
dfallo['Date'] = pd.to_datetime(dfallo['Date'], errors = 'coerce')
dfallo['Date'] = pd.to_datetime(dfallo["Date"].dt.strftime('%m/%d/%Ym'))

In [16]:
#Removing NaN, and missing (999) values from AllocaitonAmount, AllocaitonMaxium, IrrigatedAcerage.
# dfallo['AllocationFlow_CFS'] = dfallo['AllocationFlow_CFS'].fillna(0)
# dfallo['AllocationFlow_CFS'] = dfallo['AllocationFlow_CFS'].replace(999, 0)

# dfallo['AllocationVolume_AF'] = dfallo['AllocationVolume_AF'].fillna(0)
# dfallo['AllocationVolume_AF'] = dfallo['AllocationVolume_AF'].replace(999, 0)

In [17]:
#AllocationOwner
AllocationOwnerDic = {
"DREAMWORK MANAGEMENT INC; U.S.-FOREST SERVICE; NEL, ALLEN G." : "US Forest Service",
"FOREST SERVICE LINE SPG" : "US Forest Service",
"FOREST SERVICE SPRING PIPELINE" : "US Forest Service",
"FOREST SERVICE,UNITED STATES" : "US Forest Service",
"FOREST SERVICE,UNITED STATES OF AMERICA" : "US Forest Service",
"Forest Service,United States of America -" : "US Forest Service",
"FOREST SERVICE,USDA" : "US Forest Service",
"MAGDALENA RANGER DISTRICT,US Forest Service" : "US Forest Service",
"N.D. Forest Service" : "US Forest Service",
"Rowser and et al.,Lillian C.,USA Forest Service,Rasmussen,Roger J." : "US Forest Service",
"Rowser and et al.,Lillian C.,USA Forest Service,Rasmusson,Roger J.,Hood,E. Ward & Linnea" : "US Forest Service",
"SANTA FE NATIONAL FOREST,USDA FOREST SERVICE" : "US Forest Service",
"State of Utah Division of Wildlife Resources,USA Forest Service" : "US Forest Service",
"Steed,Newel,USA Forest Service" : "US Forest Service",
"TEXAS A&M FOREST SERVICE" : "US Forest Service",
"U S FOREST SERVICE" : "US Forest Service",
"U S Forest Service" : "US Forest Service",
"U. S. FOREST SERVICE" : "US Forest Service",
"U. S. Forest Service, Lassen National Forest, Hat Creek Ranger District" : "US Forest Service",
"U.S. Forest Service" : "US Forest Service",
"U.S. FOREST SERVICE" : "US Forest Service",
"U.S. FOREST SERVICE" : "US Forest Service",
"U.S. FOREST SERVICE; APPLEGATE RANGER DISTRICT" : "US Forest Service",
"U.S. FOREST SERVICE; BEND RANGER DISTRICT" : "US Forest Service",
"U.S. FOREST SERVICE; COLUMBIA RIVER GORGE SCENIC AREA" : "US Forest Service",
"U.S. FOREST SERVICE; MIDDLE FORK RANGER DISTRICT" : "US Forest Service",
"U.S.-FOREST SERVICE" : "US Forest Service",
"U.S.-FOREST SERVICE; U.S.-AGRICULTURE DEPARTMENT" : "US Forest Service",
"UNITED STATES FOREST SERVICE" : "US Forest Service",
"UNITED STATES FOREST SERVICE" : "US Forest Service",
"United States Of America Acting Through Usda Forest Service" : "US Forest Service",
"UNITED STATES OF AMERICA FOREST SERVICE" : "US Forest Service",
"UNITED STATES OF AMERICA, FOREST SERVICE" : "US Forest Service",
"US Forest Service" : "US Forest Service",
"US Forest Service" : "US Forest Service",
"US Forest Service Mount Baker," : "US Forest Service",
"US Forest Service NW International," : "US Forest Service",
"US Forest Service Snoqualmie," : "US Forest Service",
"US Forest Service Wenatchee," : "US Forest Service",
"US Forest Service," : "US Forest Service",
"US Forest Service, LOS PADRES NATIONAL FOREST" : "US Forest Service",
"USA FOREST SERVICE" : "US Forest Service",
"USA Forest Service" : "US Forest Service",
"USA Forest Service - Dixie National Forest" : "US Forest Service",
"USA Forest Service - Dixie National Service" : "US Forest Service",
"USA Forest Service - Fishlake National Forest" : "US Forest Service",
"USA Forest Service / Fishlake National Forest" : "US Forest Service",
"USA Forest Service,Ackerman,Wilford" : "US Forest Service",
"USA Forest Service,Dixie National Forest,USA FOREST SERVICE" : "US Forest Service",
"USA Forest Service,Gardner,Alan Dean,Gardner,Dean C.,Burgess,C. Judd,Gardner,Larry H." : "US Forest Service",
"USA Forest Service,Heirs of E. J. Hancock" : "US Forest Service",
"USA Forest Service,Jones,Richard N." : "US Forest Service",
"USA Forest Service,USA Forest Service 2" : "US Forest Service",
"USA Forest Service,USA Forestt Service" : "US Forest Service",
"USA Forest Service,USA Forsest Service" : "US Forest Service",
"USA Forest Service." : "US Forest Service",
"USA National Forest 2,USA Forest Service" : "US Forest Service",
"USA USDA FOREST SERVICE" : "US Forest Service",
"USDA FOREST SERVICE" : "US Forest Service",
"USDA Forest Service" : "US Forest Service",
"USDA FOREST SERVICE CIBOLA NF" : "US Forest Service",
"USDA FOREST SERVICE LAKE TAHOE BASIN MANAGEMENT UNIT" : "US Forest Service",
"USDA FOREST SERVICE(CARSON NF)" : "US Forest Service",
"USDA Forest Service," : "US Forest Service",
"USDA FOREST SERVICE, SEQUOIA NATIONAL FOREST" : "US Forest Service",
"USDA Forest Service, USA" : "US Forest Service",
"USDA FOREST SERVICE,CARSON NF" : "US Forest Service",
"USDA FOREST SERVICE; DESCHUTES NATIONAL FOREST" : "US Forest Service",
"USDA FOREST SERVICES" : "US Forest Service",
"USDA, FOREST SERVICE,CARSON NF" : "US Forest Service",
"West Panguitch Irrigation and Reservoir Company,USA Forest Service" : "US Forest Service",
"Department of Energy,United States of America" : "US Department of Energy",
"M-K Ferguson Co. for USA Department of Energy" : "US Department of Energy",
"MK-Ferguson Co. for the USA Department of Energy" : "US Department of Energy",
"U S DEPARTMENT OF ENERGY" : "US Department of Energy",
"U.S. Department of Energy" : "US Department of Energy",
"U.S.-DEPARTMENT OF ENERGY,  YUCCA MOUNTAIN SITE CHARACTERIZATION PROJECT" : "US Department of Energy",
"U.S-DEPARTMENT OF ENERGY, YUCCA MOUNTAIN SITE CHARACTERIZATION PROJECT" : "US Department of Energy",
"UNITED STATES DEPARTMENT OF ENERGY" : "US Department of Energy",
"United States of America,U.S. Department of Energy" : "US Department of Energy",
"US Department of Energy," : "US Department of Energy",
"USA Department of Energy Grand Junction Projects" : "US Department of Energy",
"Bankhead,S. Clair,UT Reclamation Mitigation and Conservation Comm,United States of America Bureau of Reclamation,Corbridge,William B." : "US Bureau Reclamation",
"Bryan, Timothy E & Jennifer,Fletcher, Gary & Carla,Teanaway Associates,,Perkins, Charles J & Catherine,US Bureau Reclamation - Yakima," : "US Bureau Reclamation",
"Bureau Of Reclamation" : "US Bureau Reclamation",
"BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"Bureau Of Reclamation," : "US Bureau Reclamation",
"Bureau of Reclamation, Provo Area Office,United States of America,Richardson,Floyd C.,Richardson,W. Earl,Richardson,Jay V.,Sandy City,Richardson,Duane G." : "US Bureau Reclamation",
"Bureau of Reclamation,The Secretary of the Interior" : "US Bureau Reclamation",
"Bureau of Reclamation,United State of America" : "US Bureau Reclamation",
"BUREAU OF RECLAMATION,UNITED STATES D.O.I." : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America" : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America," : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America,Corbridge,William B." : "US Bureau Reclamation",
"BUREAU OF RECLAMATION; UMATILLA FIELD OFFICE" : "US Bureau Reclamation",
"Heart K Ranch,,US Bureau Reclamation - Yakima," : "US Bureau Reclamation",
"Hirst,Paul and Bonnie,Weber River Water Users Association,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Lambert,Paul,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Morgan County,Weber River Water Users` Assocation,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Provo Area Office,U.S.A. Bureau of Reclamation" : "US Bureau Reclamation",
"Provo Area Office,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Remund,Glen & Unice,United States of America Bureau of Reclamation" : "US Bureau Reclamation",
"Smith,Clarence,Siddoway,Benjamin and Beryl,Vernon,James,Vernon,Francis,Gines,Mark E. and Marianne A.,USA Bureau of Reclamation,Smith,Dick,Siddoway,Alan Siddoway and LouDean,Crandall Creek Irrigation Company" : "US Bureau Reclamation",
"U S BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U. S. DEPT. OF THE INTERIOR,BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. Bureau Of Reclamation" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S.A. Bureau of Reclamation" : "US Bureau Reclamation",
"U.S.A. Bureau of Reclamation,Fitzgerald,Jackson R." : "US Bureau Reclamation",
"U.S.-BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"United States Bureau of Reclamation" : "US Bureau Reclamation",
"United States of America Bureau of Reclamation,Spencer Todd and Kelly Colleen Adamson Family Trust" : "US Bureau Reclamation",
"United States of America, Bureau of Reclamation" : "US Bureau Reclamation",
"US BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"US Bureau of Reclamation" : "US Bureau Reclamation",
"US Bureau of Reclamation - Boise," : "US Bureau Reclamation",
"US Bureau Of Reclamation," : "US Bureau Reclamation",
"USA Bureau of Reclamation" : "US Bureau Reclamation",
"USA Bureau of Reclamation -- Provo Area Office" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Boyden,John" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Ivie,James" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Miller,Max" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Weber River Water Users Association,Sanders Properties" : "US Bureau Reclamation",
"UT Reclamation Mitigation and Conservation Comm,United States of America Bureau of Reclamation,Bankhead,S. Clair,Corbridge,William B" : "US Bureau Reclamation",
"Weber River Water Users` Association,Morgan County,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Wilde Revocable Trust,Debra A.,Wilde Land and Livestock Company,Wilde `A` Ditch Company,Faddis,Andrew,Wilde,Henry,Wilson,John,Wilde,Richard & Betty,USA Bureau of Reclamation,Beech,Thomas L.,Wilde Revocable Trust,David Elliot,Wilde,Mary" : "US Bureau Reclamation",
"Wilkinson,Allen,United States of America Bureau of Reclamation" : "US Bureau Reclamation",
"Allen,T. L.,USA Bureau of Reclamation,Eldredge,Edmund,Robinson,Steve B. and Julie Ann,North Chalk Creek Irrigation Company" : "US Bureau Reclamation",
" - Cedar City Field Office,US Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,US Department of Interior, Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,US Department of the Interior, Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,USDI Bureau of Land Management" : "US Bureau of Land Management",
"#NAME?,US Department of Interior, Bureau of Land Management" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"Bureau of Land Management" : "US Bureau of Land Management",
"Bureau of Land Management," : "US Bureau of Land Management",
"Bureau of Land Management, Bakersfield" : "US Bureau of Land Management",
"Bureau of Land Management,Box Elder County" : "US Bureau of Land Management",
"Bureau of Land Management,Davis,Matthew W Davis (II) and Shawnee,Tebbs Living Trust,Richard Ray and Irene H." : "US Bureau of Land Management",
"Bureau of Land Management,Flinspach,Richard and Colleen" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,SOCORRO FIELD OFFICE" : "US Bureau of Land Management",
"Bureau of Land Management,Stuntz Valley Ranch LC" : "US Bureau of Land Management",
"Bureau of Land Management,T Y Cattle Company" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES GOVERNMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES OF AMERICA" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPARTMENT OF INTERIOR" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPT LAND MANAGEMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPT OF INTERIOR" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,USDI" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT; OWYHEE FIELD OFFICE" : "US Bureau of Land Management",
"Carbon County,USA Bureau of Land Management" : "US Bureau of Land Management",
"Cedar City Field Office,USDI Bureau of Land Management;" : "US Bureau of Land Management",
"Chew Livestock Inc,Bureau of Land Management" : "US Bureau of Land Management",
"Department of Interior Bureau of Land Management" : "US Bureau of Land Management",
"Department of Natural Resources,State of Utah,,USA Bureau of Land Management" : "US Bureau of Land Management",
"Division of Wildlife Resources,State of Utah,,USA Bureau of Land Management,Price Field Office" : "US Bureau of Land Management",
"Gemstone Mining Inc.,USA Bureau of Land Management" : "US Bureau of Land Management",
"Grouse Creek Ranch LC,USA Bureau of Land Management" : "US Bureau of Land Management",
"Jordan,W. Clift,USA Bureau of Land Management" : "US Bureau of Land Management",
"Materials Energy Research and Recovery,USA Bureau of Land Management" : "US Bureau of Land Management",
"Moab District USA Bureau of Land Management" : "US Bureau of Land Management",
"Nelson,Rick M.,USA Bureau of Land Management" : "US Bureau of Land Management",
"NORTHERN WASHOE RANCHING MANAGEMENT, LLC; DUCK LAKE RANCH, LLC; U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"NORTHERN WASHOE RANCHING MANAGEMENT, LLC; U.S.-BUREAU OF LAND MANAGEMENT; DUCK LAKE RANCH, LLC" : "US Bureau of Land Management",
"Richfield District USA Bureau of Land Management" : "US Bureau of Land Management",
"Saint George Field Office,USDI Bureau of Land Management" : "US Bureau of Land Management",
"Saint George Field Office,USDI Bureau of Land Management;" : "US Bureau of Land Management",
"Skull Valley Company Ltd.,USA Bureau of Land Management" : "US Bureau of Land Management",
"Staker,James Allen,USA Bureau of Land Management" : "US Bureau of Land Management",
"State of Utah Division of Wildlife Resources,USA Bureau of Land Management" : "US Bureau of Land Management",
"Thousand Peaks Ranches, Inc.,USA Bureau of Land Management,Richfield District" : "US Bureau of Land Management",
"U S BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S. Bureau Of Land Management" : "US Bureau of Land Management",
"U.S. BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S. DEPARTMENT OF THE INTERIOR; BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT; BRIGHT-HOLLAND CORPORATION" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT; WHITE PINE LUMBER CO." : "US Bureau of Land Management",
"U.S.D.I., Bureau of Land Management" : "US Bureau of Land Management",
"United States Of America Acting Through The Department Of Interior Bureau Of Land Management" : "US Bureau of Land Management",
"United States of America Bureau of Land Management,Fillmore District" : "US Bureau of Land Management",
"UNITED STATES OF AMERICA, DEPARTMENT OF THE INTERIOR, BUREAU OF LAND MANAGEMENT; WULFENSTEIN TRUST R-501; SPRING MOUNTAIN MOTORSPORTS RANCH; PAHRUMP GATEWAY UTILITY; SPRING MOUNTAIN RACEWAY LLC" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT / BARSTOW FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT / PALM SPRINGS FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, ALTURAS FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, RIDGECREST FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, UKIAH FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT/ ARCATA FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT/NEEDLES FIELD OFFICE" : "US Bureau of Land Management",
"US DEPT OF THE INTERIOR,BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"USA Bureau of Land Management" : "US Bureau of Land Management",
"USA Bureau of Land Management - Filmore Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management (Richfield District)" : "US Bureau of Land Management",
"USA Bureau of Land Management Vernal District" : "US Bureau of Land Management",
"USA Bureau of Land Management, Richfield District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Cedar City District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Fillmore District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Fillmore Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Kanab District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Kanab Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Moab District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Monticello  District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Monticello District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Ogden District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Price District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Price Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Richfield District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Vernal District" : "US Bureau of Land Management",
"USDI BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"USDI BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"Utah School and Institutional Trust Lands Admin.,USA Bureau of Land Management" : "US Bureau of Land Management",
"West Panguitch Irrigation and Reservoir Company,State of Utah Board of Water Resources,USA Bureau of Land Management" : "US Bureau of Land Management",
"WHITE PINE LUMBER CO; U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"ARMY CAMP SPRING" : "US Army",
"CORPS OF ENGRS. ABIQUIU DAM,US ARMY ENGR DIST ALBQ." : "US Army",
"Cummins, Dale F,US Army Corps Of Engineers," : "US Army",
"Dept Of The Army Corps Of Engineers," : "US Army",
"LONGHORN ARMY AMMUNITION PLANT,U.S. DEPARTMENT OF THE INTERIOR FISH AND WILDLIFE SERVICE" : "US Army",
"U S ARMY CORPS OF ENGINEERS" : "US Army",
"U S Army Corps of Engineers" : "US Army",
"U S Army, Department of" : "US Army",
"U.S Army Corps Of Engineers" : "US Army",
"U.S. ARMY CORP OF ENGINEERS" : "US Army",
"U.S. Army Corps Of Engineers" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS SEATTLE" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS WALLA WALLA" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS, WALLA WALLA DISTRICT" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS; PORTLAND DISTRICT" : "US Army",
"U.S. ARMY CORPS OF ENGINEERSSEATTLE" : "US Army",
"U.S. ARMY CORPS OF ENGINEERSWALLA WALLA" : "US Army",
"U.S. Army Dugway Proving Ground" : "US Army",
"U.S. ARMY WHITE SANDS MISSILE" : "US Army",
"U.S. ARMY-WHITE SANDS MISSILE" : "US Army",
"UNITED STATES ARMY CORPS OF ENGINEERS" : "US Army",
"US ARMY" : "US Army",
"Us Army Corps Of Engineers" : "US Army",
"US ARMY CORPS OF ENGINEERS" : "US Army",
"US Army Corps Of Engineers," : "US Army",
"US ARMY CORPS OF ENGINGEERS" : "US Army",
"US DEPARTMENT OF THE ARMY" : "US Army",
"USA Army Dugway Proving Ground" : "US Army",
"USA Army Dugway Proving Grounds" : "US Army",
"USA Army Engineer District Corps of Engineers" : "US Army",
"USA Army for Dugway Proving Ground" : "US Army",
"USA Army Ogden Sub-Depot of Tooele Ordinance Depot" : "US Army",
"USA Army Tooele Army Depot South" : "US Army",
"USA Department of the Army" : "US Army",
"USA Department of the Army - Dugway Proving Ground" : "US Army",
"USA Department of the Army - Tooele Army Depot" : "US Army",
"USA Department of the Army Arsenal" : "US Army",
"USARMY Corp Of Engineers," : "US Army",
"USARMY Corps Engineers Columbia Cnty," : "US Army",
"USARMY Corps Engineers Seattle," : "US Army",
"USARMY Corps Engineers Walla Walla," : "US Army",
"USARMY Corps Engineers," : "US Army",
"USARMY Corps Engineers,,USDA Agricultural Research Service WRPIS," : "US Army",
"WA Fish & Wildlife Dept - ERO,,USARMY Corps Engineers," : "US Army",
"Camelot International Resort Corporation,United States of America" : "United States of America",
"Christensen,Mark B.,Fish and Wildlife Service,United States of America" : "United States of America",
"CIBOLA NATIONAL FOREST,UNITED STATES OF AMERICA," : "United States of America",
"Ekker,Sheena,Utah School and Institutional Trust Lands Admin.,Hodges,Jimmy,McArthur,Ty,United States of America" : "United States of America",
"Fish and Wildlife Service,United States of America" : "United States of America",
"Fish and Wildlife Service,United States of America," : "United States of America",
"INTERMOUNTAIN LAND CO.; MOONROCKS GATEWAY LLC; UNITED STATES OF AMERICA" : "United States of America",
"LAMB, SHELDON W. & ISABEL H.; UNITED STATES OF AMERICA" : "United States of America",
"Llewellyn,Robert W.,United States of America" : "United States of America",
"R. CLYDE HIIBEL FAMILY TRUST; UNITED STATES OF AMERICA" : "United States of America",
"The United States of America" : "United States of America",
"THE UNITED STATES OF AMERICA IN TRUST FOR THE PYRAMID LAKE PAIUTE TRIBE; GARAVANTA LAND AND LIVESTOCK COMPANY SUCCESSOR TO J. GARAVANTA" : "United States of America",
"THOMAS, DARRELL E.; THOMAS, BEVERLY J.; UNITED STATES OF AMERICA" : "United States of America",
"United States Of America" : "United States of America",
"UNITED STATES OF AMERICA" : "United States of America",
"UNITED STATES OF AMERICA" : "United States of America",
"United States of America" : "United States of America",
"UNITED STATES OF AMERICA (BLM)" : "United States of America",
"United States Of America Acting Through" : "United States of America",
"United States of America in trust for Raymond Murray,Burton,John Ira" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR SUCH SHOSHONE INDIANS SOUTHERN NEVADA" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR THE WASHOE TRIBE OF INDIANS" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR YOMBA TRIBE OF SHOSHONE INDIANS" : "United States of America",
"UNITED STATES OF AMERICA NAS" : "United States of America",
"UNITED STATES OF AMERICA NAVAL AIR STATION FALLON" : "United States of America",
"United States Of America Trustee" : "United States of America",
"United States Of America," : "United States of America",
"UNITED STATES OF AMERICA,  C/O COMMANDER AND DISTRICT ENGINEER,   UNITED STATES ARMY ENGINEER  DISTRICT, LOUISVILLE" : "United States of America",
"United States of America, Utah" : "United States of America",
"United States of America,BBRD, LC,Ottesen,Nellie" : "United States of America",
"United States of America,Owen Probst and Rosetta B. Probst Trust" : "United States of America",
"United States of America,The,Conger,Wm. B. and Florance H" : "United States of America",
"United States of America,Virgin River Land Preservation Inc." : "United States of America",
"UNITED STATES OF AMERICA; STREETS; UNITED STATES OF AMERICA IN TRUST FOR THE RENO SPARKS INDIAN COLONY; NEVADA-DEPARTMENT OF TRANSPORTATION; SIERRA PACIFIC POWER COMPANY; RENO SPARKS INDIAN COLONY; RENO LODGE #13 F AND A M" : "United States of America",
"United States of American" : "United States of America",
"USA-United States Of America" : "United States of America",
"Utah Reclamation Mitigation and Conservation Commission,United States Of America" : "United States of America",
"WASHOE COUNTY; PRATT, TOM & LANNETTE; UNITED STATES OF AMERICA" : "United States of America",
"WS-BUCKLAND, LLC; UNITED STATES OF AMERICA" : "United States of America"}
 
def assignAllocationOwner(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "Other"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = AllocationOwnerDic[String1]
        except:
            outList = "Other"
    return outList

dfallo['AllocationOwner'] = dfallo.apply(lambda row: assignAllocationOwner(row['AllocationOwner']), axis=1)

In [18]:
#Sort by AllocationAmountID and SiteUUID
dfallo = dfallo.sort_values(by=['AllocationAmountID', 'SiteUUID'], ascending=True).reset_index()
dfallo

,index,AllocationAmountID,AllocationNativeID,AllocationFlow_CFS,AllocationVolume_AF,AllocationOwner,Date,SiteUUID,SiteTypeCV,Longitude,...,WaterSourceID,WaterSourceUUID,WaterSourceName,WaterSourceTypeCV,OrganizationID,OrganizationUUID,State,WaDENameBenUse,WaDENameWS,WaDENameS
0,461254,2734281,50769.493780000004-0-C-100503,150.000000,0.0,Other,1985-03-11,CO_2,Ditch,-104.468875,...,38325,CO_1,SOUTH PLATTE RIVER,Surface Water,1,CODWR,CO,Unknown,Surface Water,Canal / Ditch / Stream
1,404066,2734282,51356.0-0-C-100503,120.000000,0.0,Other,1990-08-10,CO_2,Ditch,-104.468875,...,38325,CO_1,SOUTH PLATTE RIVER,Surface Water,1,CODWR,CO,Storage,Surface Water,Canal / Ditch / Stream
2,331198,2734283,53330.0-0-C-100503,1000.000000,0.0,Other,1996-01-05,CO_2,Ditch,-104.468875,...,38325,CO_1,SOUTH PLATTE RIVER,Surface Water,1,CODWR,CO,Unknown,Surface Water,Canal / Ditch / Stream
3,456022,2734284,55637.0-0-C-100503,358.000000,0.0,Other,2002-04-30,CO_2,Ditch,-104.468875,...,38325,CO_1,SOUTH PLATTE RIVER,Surface Water,1,CODWR,CO,Unknown,Surface Water,Canal / Ditch / Stream
4,393549,2734285,55989.0-0-C-100503,1080.000000,0.0,Other,2003-04-17,CO_2,Ditch,-104.468875,...,38325,CO_1,SOUTH PLATTE RIVER,Surface Water,1,CODWR,CO,Unknown,Surface Water,Canal / Ditch / Stream
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699639,305145,6542174,S017140,NaN,0.0,Other,2011-07-19,CAwr_S40627,Unspecified,-121.898700,...,737421,CAwr_WS6644,SPOON BILL SLOUGH AND SACRAMENTO RIVER,Unspecified,17,CSWRCB,CA,Agricultural,Unknown,Other
1699640,323088,6542175,S021201,NaN,0.0,Other,2012-07-27,CAwr_S44713,Unspecified,-121.915900,...,737547,CAwr_WS6770,MONTEZUMA SLOUGH,Surface,17,CSWRCB,CA,Fish,Surface Water,Other
1699641,312193,6542176,S028209,0.445602,NaN,Other,2019-08-16,CAwr_S53721,Other,-120.497520,...,740294,CAwr_WS9517,Dorsey Creek,Surface,17,CSWRCB,CA,Environmental,Surface Water,Other
1699642,303096,6542177,S010383,NaN,0.0,Other,1981-06-02,CAwr_S33517,Unspecified,-121.856928,...,733509,CAwr_WS2732,SINGER CREEK,Unspecified,17,CSWRCB,CA,Domestic,Unknown,Other


In [19]:
## Create Allo Dataframe - with attached Site and Benuse info ##
########################################
outdfallo = pd.DataFrame()

#The Columns
outdfallo['AA_ID'] = dfallo['AllocationAmountID'].astype(str) 
outdfallo['SiteUUID'] = dfallo['SiteUUID'].astype(str)
outdfallo['SN_ID'] = dfallo['AllocationNativeID'].astype(str) 
outdfallo['Lat'] = dfallo['Latitude'].astype(float) 
outdfallo['Long'] = dfallo['Longitude'].astype(float)
outdfallo['WBenUse'] = dfallo['WaDENameBenUse'].astype(str)

outdfallo['State'] = dfallo['State'].astype(str) 

outdfallo['AA_CFS'] = dfallo['AllocationFlow_CFS'].astype(float) 
outdfallo['AA_AF'] = dfallo['AllocationVolume_AF'].astype(float) 
outdfallo['PD'] = dfallo['Date']
outdfallo['WSN'] = dfallo['WaterSourceName'].astype(str)
outdfallo['SiteTypeCV'] = dfallo['WaDENameS'].astype(str)
outdfallo['WaterSourceTypeCV'] = dfallo['WaDENameWS'].astype(str)
outdfallo['AllocationOwner'] = dfallo['AllocationOwner'].astype(str)

outdfallo

,AA_ID,SiteUUID,SN_ID,Lat,Long,WBenUse,State,AA_CFS,AA_AF,PD,WSN,SiteTypeCV,WaterSourceTypeCV,AllocationOwner
0,2734281,CO_2,50769.493780000004-0-C-100503,40.379142,-104.468875,Unknown,CO,150.000000,0.0,1985-03-11,SOUTH PLATTE RIVER,Canal / Ditch / Stream,Surface Water,Other
1,2734282,CO_2,51356.0-0-C-100503,40.379142,-104.468875,Storage,CO,120.000000,0.0,1990-08-10,SOUTH PLATTE RIVER,Canal / Ditch / Stream,Surface Water,Other
2,2734283,CO_2,53330.0-0-C-100503,40.379142,-104.468875,Unknown,CO,1000.000000,0.0,1996-01-05,SOUTH PLATTE RIVER,Canal / Ditch / Stream,Surface Water,Other
3,2734284,CO_2,55637.0-0-C-100503,40.379142,-104.468875,Unknown,CO,358.000000,0.0,2002-04-30,SOUTH PLATTE RIVER,Canal / Ditch / Stream,Surface Water,Other
4,2734285,CO_2,55989.0-0-C-100503,40.379142,-104.468875,Unknown,CO,1080.000000,0.0,2003-04-17,SOUTH PLATTE RIVER,Canal / Ditch / Stream,Surface Water,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699639,6542174,CAwr_S40627,S017140,38.058100,-121.898700,Agricultural,CA,NaN,0.0,2011-07-19,SPOON BILL SLOUGH AND SACRAMENTO RIVER,Other,Unknown,Other
1699640,6542175,CAwr_S44713,S021201,38.134400,-121.915900,Fish,CA,NaN,0.0,2012-07-27,MONTEZUMA SLOUGH,Other,Surface Water,Other
1699641,6542176,CAwr_S53721,S028209,39.604640,-120.497520,Environmental,CA,0.445602,NaN,2019-08-16,Dorsey Creek,Other,Surface Water,Other
1699642,6542177,CAwr_S33517,S010383,39.984789,-121.856928,Domestic,CA,NaN,0.0,1981-06-02,SINGER CREEK,Other,Unknown,Other


In [20]:
#Exporting Outbound DataFrame to working csv file.
outdfallo.to_csv('P_AlloLJSite.csv', index=False)  # The output

# Checking Some Data

In [21]:
sitecheck = outdfSite
sitecheckB = sitecheck[(sitecheck.State == 'WY')].sort_values(by=['SiteUUID'])
sitecheckB

,SiteUUID,Lat,Long,WBenUse,WBenUseColor,SiteTypeColor,State,WaterSourceTypeColor,OwnerColor
1266993,WYwr_S1,42.912900,-104.461270,Agricultural,#006400FF,#6495EDFF,WY,#008000FF,#D3D3D3FF
1266994,WYwr_S10,43.097970,-105.562920,Industrial,#800080FF,#6495EDFF,WY,#008000FF,#D3D3D3FF
1266995,WYwr_S100,43.550178,-105.250300,Other,#808080FF,#6495EDFF,WY,#008000FF,#D3D3D3FF
1266996,WYwr_S1000,42.507690,-104.409110,Agricultural,#006400FF,#6495EDFF,WY,#008000FF,#D3D3D3FF
1266997,WYwr_S10000,41.231404,-109.146703,Livestock,#FFD700FF,#00FFFFFF,WY,#FF4500FF,#D3D3D3FF
...,...,...,...,...,...,...,...,...,...
1277107,WYwr_S9995,41.410619,-109.019069,Unknown,#D3D3D3FF,#00FFFFFF,WY,#FF4500FF,#D3D3D3FF
1277108,WYwr_S9996,41.281844,-109.338400,Livestock,#FFD700FF,#00FFFFFF,WY,#FF4500FF,#D3D3D3FF
1277109,WYwr_S9997,41.387744,-108.522636,Livestock,#FFD700FF,#00FFFFFF,WY,#FF4500FF,#D3D3D3FF
1277110,WYwr_S9998,41.307778,-109.094578,Unknown,#D3D3D3FF,#00FFFFFF,WY,#FF4500FF,#D3D3D3FF


In [22]:
allocheck = outdfallo
allocheckB = allocheck[(allocheck.State == 'WY')].sort_values(by=['SiteUUID'])
allocheckB

,AA_ID,SiteUUID,SN_ID,Lat,Long,WBenUse,State,AA_CFS,AA_AF,PD,WSN,SiteTypeCV,WaterSourceTypeCV,AllocationOwner
951551,5582332,WYwr_S1,CR UW04/059,42.912900,-104.461270,Agricultural,WY,575.00,NaN,1976-08-05,Unspecified,Well,Groundwater,Other
952138,5582919,WYwr_S10,P114747.0W,43.097970,-105.562920,Industrial,WY,1110.00,NaN,1999-03-31,Unspecified,Well,Groundwater,Other
951255,5582036,WYwr_S100,P189568.0W,43.550178,-105.250300,Other,WY,500.00,NaN,2008-10-20,Unspecified,Well,Groundwater,Other
954424,5585206,WYwr_S1000,CR UW01/482,42.507690,-104.409110,Agricultural,WY,675.00,NaN,1940-10-10,Unspecified,Well,Groundwater,Other
956174,5586960,WYwr_S10000,CR CC27/331,41.231404,-109.146703,Livestock,WY,0.00,NaN,1906-09-12,Springs on Head of Mellor Creek,Spring,Surface Water,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950641,5581422,WYwr_S9995,P8093.0D,41.410619,-109.019069,Unknown,WY,0.00,NaN,1908-01-04,Snow Spring,Spring,Surface Water,Other
949765,5580546,WYwr_S9996,CR CB03/174,41.281844,-109.338400,Livestock,WY,1.00,NaN,1882-12-31,Spring Creek,Spring,Surface Water,Other
951662,5582443,WYwr_S9997,CR CC25/113,41.387744,-108.522636,Livestock,WY,3.11,NaN,1905-11-11,"Antelope Springs 1, 2, & 3",Spring,Surface Water,Other
954082,5584864,WYwr_S9998,CR CC27/327,41.307778,-109.094578,Unknown,WY,0.35,NaN,1904-11-09,Spring on Head of Pretty Water Creek,Spring,Surface Water,Other


In [24]:
allocheck = outdfallo
allocheckB = allocheck['WaterSourceTypeCV'].drop_duplicates()
allocheckB

0               Surface Water
965               Groundwater
1348                  Unknown
1887      Reservoir / Storage
174624                  Other
817266                    nan
Name: WaterSourceTypeCV, dtype: object